In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [8]:
df = pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\old\data\\1h\ARUSDT_1h.csv", index_col='Time')
df.index = pd.to_datetime(df.index)

In [19]:
# williams %R
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

df['%R14'] = get_wr(df['High'], df['Low'], df['Close'], 14)



In [ ]:
def get_macd(price, fast, slow, smooth):
    exp1 = price.ewm(span = fast, adjust=False).mean()
    exp2 = price.ewm(span = slow, adjust=False).mean()
    macd = pd.DataFrame(exp1-exp2).rename(columns={'Close':'macd'})
    
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist
    
get_macd(df['Close'], 26, 12, 9)[1]

In [30]:
df['macd'] = get_macd(df['Close'], 26, 12, 9)[0]
df['macd_signal'] = get_macd(df['Close'], 26, 12, 9)[1]
df['macd_hist'] = get_macd(df['Close'], 26, 12, 9)[2]
df = df.dropna()

In [31]:
df

,Open,High,Low,Close,Volume,%R14,macd,macd_signal,macd_hist
Time,,,,,,,,,
2022-03-15 07:00:00,30.02,30.06,29.63,29.78,29048.89,-91.017964,0.012675,-0.047483,0.060159
2022-03-15 08:00:00,29.77,29.80,29.65,29.67,9412.07,-97.604790,0.049301,-0.028126,0.077428
2022-03-15 09:00:00,29.67,30.01,29.63,29.99,19894.90,-78.443114,0.051908,-0.012119,0.064028
2022-03-15 10:00:00,30.00,30.52,29.99,30.43,16135.76,-52.095808,0.018259,-0.006044,0.024303
2022-03-15 11:00:00,30.45,30.45,29.95,30.14,11924.76,-69.461078,0.014822,-0.001871,0.016692
...,...,...,...,...,...,...,...,...,...
2022-04-25 03:00:00,26.22,26.60,26.22,26.58,3673.61,-66.666667,0.466637,0.356765,0.109872
2022-04-25 04:00:00,26.58,26.58,26.42,26.45,2153.11,-72.361809,0.466623,0.378737,0.087886
2022-04-25 05:00:00,26.45,26.61,26.23,26.26,6936.40,-80.952381,0.476451,0.398280,0.078172
